In [1]:
import requests
import tarfile
from os import path

In [3]:
# Downloading the dataset

fname = 'airline_2m.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)


151681776

In [4]:
# Extracting the dataset
tar = tarfile.open(fname)
tar.extractall()
tar.close()


In [5]:
# Verifying the file was extracted properly
data_path = "airline_2m.csv"
path.exists(data_path)

True

In [6]:
import pandas as pd 

In [7]:
df = pd.read_csv(data_path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})

In [8]:
df.shape

(2000000, 109)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Columns: 109 entries, Year to Div5TailNum
dtypes: float64(72), int64(18), object(19)
memory usage: 1.6+ GB


In [13]:
df1=pd.read_csv('cases.csv')
df1

,Column,Unique,Count,Categorical?,Needed?
0,Year,34,2000000,1.0,1.0
1,Quarter,4,2000000,1.0,1.0
2,Month,12,2000000,1.0,1.0
3,DayofMonth,31,2000000,1.0,1.0
4,DayOfWeek,7,2000000,1.0,1.0
5,DOT_ID_Reporting_Airline,34,2000000,1.0,1.0
6,Flight_Number_Reporting_Airline,8050,2000000,1.0,1.0
7,OriginAirportID,407,2000000,1.0,1.0
8,OriginState,53,1999354,1.0,1.0
9,DestAirportID,409,2000000,1.0,1.0


In [14]:
featurecols= pd.Series(df1['Column'])
featurecols

0                                Year
1                             Quarter
2                               Month
3                          DayofMonth
4                           DayOfWeek
5            DOT_ID_Reporting_Airline
6     Flight_Number_Reporting_Airline
7                     OriginAirportID
8                         OriginState
9                       DestAirportID
10                          DestState
11                            DepTime
12                          Cancelled
Name: Column, dtype: object

In [15]:
featurecols= ['Year','Quarter',
'Month','DayofMonth','DayOfWeek','DOT_ID_Reporting_Airline','Flight_Number_Reporting_Airline',
'OriginAirportID','OriginState','DestAirportID','DestState','DepTime','Cancelled']
featurecols

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DOT_ID_Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginState',
 'DestAirportID',
 'DestState',
 'DepTime',
 'Cancelled']

In [16]:
featuredata=df[featurecols]
featuredata.to_csv('featuredata.csv',index=False)

In [17]:
featuredata=pd.read_csv('featuredata.csv')
featuredata=featuredata.dropna()
featuredata

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime,Cancelled
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0,0.0
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202.0,0.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644.0,0.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305.0,0.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2008,1,3,23,7,19393,966,12889,NV,14107,AZ,1444.0,0.0
1999996,1999,1,1,5,2,19704,529,11618,NJ,11298,TX,945.0,0.0
1999997,2003,4,11,14,5,20355,1457,10994,SC,11057,NC,1219.0,0.0
1999998,2012,2,5,15,2,19393,536,13232,IL,10693,TN,1838.0,0.0


In [18]:
target_name = "Cancelled"
target = featuredata[target_name]

data = featuredata.drop(columns=[target_name])

In [19]:
target.value_counts()

0.0    1962513
1.0        457
Name: Cancelled, dtype: int64

In [20]:
data.isnull().values.any()

False

In [21]:
data

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2008,1,3,23,7,19393,966,12889,NV,14107,AZ,1444.0
1999996,1999,1,1,5,2,19704,529,11618,NJ,11298,TX,945.0
1999997,2003,4,11,14,5,20355,1457,10994,SC,11057,NC,1219.0
1999998,2012,2,5,15,2,19393,536,13232,IL,10693,TN,1838.0


In [22]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(data, target)
dummy_preds=dummy_clf.predict(data)
dummy_clf.predict(data)
dummy_clf.score(data, target)

0.9997671895138489

In [23]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

logistic_regression = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression()
)


from sklearn.model_selection import cross_validate
cv_results = cross_validate(
    logistic_regression, data, target, cv=10,return_estimator=True)

cv_results

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


{'estimator': (Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype=<class 'numpy.float64'>,
                                 handle_unknown='ignore', sparse=True)),
                  ('logisticregression',
                   LogisticRegression(C=1.0, class_weight=None, dual=False,
                                      fit_intercept=True, intercept_scaling=1,
                                      l1_ratio=None, max_iter=100,
                                      multi_class='auto', n_jobs=None,
                                      penalty='l2', random_state=None,
                                      solver='lbfgs', tol=0.0001, verbose=0,
                                      warm_start=False))],
           verbose=False), Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype

In [24]:
array=cv_results["test_score"]
array.mean()

0.999767189513849

In [26]:
logistic_regression.fit(data,target)
predictions=logistic_regression.predict(data.iloc[0:1])
predictions

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([0.])

In [ ]:
data.iloc[0:1]

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0


In [33]:
predictions=logistic_regression.predict(data)
predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [34]:
from sklearn.metrics import confusion_matrix
c_matrix=confusion_matrix(target, predictions)
c_matrix

array([[1962513,       0],
       [    457,       0]])

In [29]:
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

(1962513, 0, 457, 0)

In [30]:
precision= tp/(tp+fp)
precision

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in long_scalars
  """Entry point for launching an IPython kernel.


nan

In [31]:
c_matrix_dummy=confusion_matrix(target, dummy_preds)
tn, fp, fn, tp = c_matrix_dummy.ravel()
(tn, fp, fn, tp)

(1962513, 0, 457, 0)

In [32]:
c_matrix_=confusion_matrix(predictions, dummy_preds)
c_matrix_


array([[1962970]])

In [ ]:
y_true = [1,1,1,1,1,1,1]
y_pred = [1,1,1,1,1,1,1]
confusion_matrix(y_true, y_pred)

array([[7]])

In [ ]:
target.value_counts()

0.0    1962513
1.0        457
Name: Cancelled, dtype: int64

Class imbalance is an issue


In [35]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

# fit predictor and target 

x_ros, y_ros = ros.fit_resample(data, target)

y_ros=pd.DataFrame(y_ros)

print('Original dataset shape', target.value_counts())
print('Resample dataset shape', y_ros.value_counts())

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Original dataset shape 0.0    1962513
1.0        457
Name: Cancelled, dtype: int64
Resample dataset shape 1.0    1962513
0.0    1962513
dtype: int64


In [57]:
y_ros

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3925021,1.0
3925022,1.0
3925023,1.0
3925024,1.0


In [37]:
logistic_regression = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression()
)


cv_results = cross_validate(
    logistic_regression, x_ros, y_ros[0], cv=10,return_estimator=True)

cv_results

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


{'estimator': (Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype=<class 'numpy.float64'>,
                                 handle_unknown='ignore', sparse=True)),
                  ('logisticregression',
                   LogisticRegression(C=1.0, class_weight=None, dual=False,
                                      fit_intercept=True, intercept_scaling=1,
                                      l1_ratio=None, max_iter=100,
                                      multi_class='auto', n_jobs=None,
                                      penalty='l2', random_state=None,
                                      solver='lbfgs', tol=0.0001, verbose=0,
                                      warm_start=False))],
           verbose=False), Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype

In [38]:
array=cv_results["test_score"]
array.mean()

0.9969363260723176

In [39]:
logistic_regression.fit(x_ros,y_ros[0])
predictions=logistic_regression.predict(x_ros)
c_matrix=confusion_matrix(y_ros, predictions)
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(1951834, 10679, 0, 1962513)

In [40]:
precision= tp/(tp+fp)
precision

0.9945879569752969

In [41]:
from sklearn.ensemble import (RandomForestClassifier, StackingClassifier, GradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from  sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [61]:
df=pd.DataFrame(x_ros)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911
...,...,...,...,...,...,...,...,...,...,...,...,...
3925021,2019,2,4,19,5,19790,1206,13303,FL,10397,GA,1132
3925022,2008,4,11,18,2,20374,2584,11423,IA,12266,TX,900
3925023,2011,1,1,19,3,20378,7247,12264,VA,10397,GA,1210
3925024,2010,2,4,26,1,20304,4483,11292,CO,14869,UT,723


In [63]:
df['target']=y_ros
df

,0,1,2,3,4,5,6,7,8,9,10,11,target
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659,0.0
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202,0.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644,0.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305,0.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3925021,2019,2,4,19,5,19790,1206,13303,FL,10397,GA,1132,1.0
3925022,2008,4,11,18,2,20374,2584,11423,IA,12266,TX,900,1.0
3925023,2011,1,1,19,3,20378,7247,12264,VA,10397,GA,1210,1.0
3925024,2010,2,4,26,1,20304,4483,11292,CO,14869,UT,723,1.0


In [65]:
df2=df.sample(n=300000)
df2['target'].value_counts()

1.0    150000
0.0    150000
Name: target, dtype: int64

In [66]:
x,y = df2.drop(columns=['target']),df2['target']
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y,random_state=42)

In [67]:
estimators=[
            ("rf",RandomForestClassifier(n_estimators=10,random_state=42)),
            ("gb",GradientBoostingClassifier(n_estimators=10,random_state=42)),
            ("knn",KNeighborsClassifier(n_neighbors=5))
]

In [68]:
clf=StackingClassifier(estimators=estimators,final_estimator=LogisticRegression())

In [69]:
pipeline = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), clf
)

pipeline.fit(x_train,y_train)
pipeline.score(x_test,y_test)

0.99992

In [72]:
y[:100]

1960543    0.0
2848441    1.0
3173852    1.0
2109617    1.0
2232684    1.0
          ... 
3151383    1.0
3325683    1.0
1626397    0.0
2711785    1.0
1331662    0.0
Name: target, Length: 100, dtype: float64

In [73]:
preds=pipeline.predict(x[:10000])
c_matrix=confusion_matrix(y[:10000], preds)
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

(4984, 0, 0, 5016)

In [74]:
precision= tp/(tp+fp)
precision

1.0

In [76]:
accuracy=(tp+tn)/(tp+tn+fn+fp)
accuracy

1.0